In [3]:
!pip3 install newsapi-python

You should consider upgrading via the 'pip install --upgrade pip' command.


In [41]:
# !python3 -m spacy download en_core_web_lg

In [122]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from newsapi import NewsApiClient
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import spacy
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

In [123]:
api_key = '946472f8ad574082a31c9d0fcaf1e89d'
newsapi = NewsApiClient(api_key=api_key)

In [44]:
def crawl_news(query):
    all_results = []
    for pag in tqdm(range(1, 6)):
        pag_articles = newsapi.get_everything(q=query, sort_by='relevancy', page=pag)['articles']
        if len(pag_articles) == 0:
            break
        all_results.extend(pag_articles)
    return all_results

In [45]:
tesla_news = crawl_news('tesla')

100%|██████████| 5/5 [00:05<00:00,  1.11s/it]


In [124]:
df = pd.read_csv('BBC news dataset.csv', usecols=range(1, 3))

In [126]:
df.drop_duplicates('description', inplace=True)

In [128]:
descriptions = df['description'].values

In [129]:
def pre_process_text(text):

    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

In [130]:
processed_descriptions = []
for description in tqdm(descriptions):
    processed_descriptions.append(' '.join(pre_process_text(description)))

100%|██████████| 2128/2128 [00:03<00:00, 558.18it/s]


In [131]:
nlp = spacy.load('en_core_web_sm')

sent_vecs = {}
docs = []

for index, description in enumerate(tqdm(processed_descriptions)):
    doc = nlp(description)
    docs.append(doc)
    sent_vecs[index] = doc.vector

100%|██████████| 2128/2128 [01:17<00:00, 27.58it/s]


In [132]:
vectors = list(sent_vecs.values())

In [133]:
vectors = np.array(vectors)

In [145]:
labels_results = {}
for i in tqdm(np.arange(0.001, 1, 0.002)):
    dbscan = DBSCAN(eps=i, min_samples=5, metric='cosine').fit(vectors)
    labels_results[i] = len(pd.Series(dbscan.labels_).value_counts())

100%|██████████| 500/500 [00:30<00:00, 16.34it/s]


In [146]:
for i in np.arange(0.001, 0.05, 0.002):
    print('{}: {}'.format(i, labels_results[i]))

0.001: 1
0.003: 1
0.005: 1
0.007: 1
0.009000000000000001: 1
0.011: 4
0.013000000000000001: 6
0.015: 6
0.017: 3
0.019000000000000003: 3
0.021: 2
0.023: 2
0.025: 2
0.027000000000000003: 2
0.029: 2
0.031: 2
0.033: 2
0.035: 2
0.037000000000000005: 2
0.039: 2
0.041: 2
0.043000000000000003: 2
0.045: 2
0.047: 2
0.049: 2


In [151]:
dbscan = DBSCAN(eps=0.015, min_samples=5, metric='cosine').fit(vectors)

In [152]:
results = pd.DataFrame({
    'desc': processed_descriptions, 
    'label': dbscan.labels_
})

In [153]:
results['label'].value_counts()

-1    1672
 0     432
 2       9
 1       7
 4       4
 3       4
Name: label, dtype: int64

In [155]:
for index in results[results['label'] == 3].index:
    print(results.loc[index]['desc'])
    print('....')

force strong battlefront warm reception greeted star wars battlefront reflection ingenious innovation gameplay backtobasics approach immense nostalgia quotient geared towards online gamers based around little series allout gunfights set array locations featured hinted two blockbusting film trilogies previous star wars titles like acclaimed knights old republic jedi knight regularly impressed imaginative forays far corners franchises extensive universe use weird wonderful new characters battlefront hand wholeheartedly revisits recognisable elements hit movies sights sounds protagonists show instantly familiar fans may well feel opportunity relive star wars memorable screen skirmishes makes game always waited mayhem viewed either third firstperson perspective either fight forces freedom join darth vader dark side depending episode type campaign well players personal propensity good evil ample chance wookie shoot ewoks rush battle alongside firedup luke skywalker section task simply wipe 